<a href="https://colab.research.google.com/github/yamini542/Applied-AI/blob/main/AI_Time_series_Exercise1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Exercise:
Build an appropriate ARMA model to predict the number of international airline passengers during the months in the first quarter of 1961 using the dataset international-airline-passengers.csv



In [1]:
#import necessary libraries
import statsmodels.api as sm 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker 
import statsmodels.api as sm
import itertools
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
from scipy.spatial.distance import cdist


In [5]:
#loading the data set from the github

df= pd.read_csv('https://raw.githubusercontent.com/HHansi/Applied-AI-Course/main/Time%20Series%20Analysis/data/international-airline-passengers.csv')


In [6]:
df.head

<bound method NDFrame.head of        Month  \
0    1949-01   
1    1949-02   
2    1949-03   
3    1949-04   
4    1949-05   
..       ...   
139  1960-08   
140  1960-09   
141  1960-10   
142  1960-11   
143  1960-12   

     International airline passengers: monthly totals in thousands. Jan 49 ? Dec 60  
0                                                  112                               
1                                                  118                               
2                                                  132                               
3                                                  129                               
4                                                  121                               
..                                                 ...                               
139                                                606                               
140                                                508                               
141

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 2 columns):
 #   Column                                                                          Non-Null Count  Dtype 
---  ------                                                                          --------------  ----- 
 0   Month                                                                           144 non-null    object
 1   International airline passengers: monthly totals in thousands. Jan 49 ? Dec 60  144 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ KB


In [8]:
#rename the columns
df=df.rename(columns={"International airline passengers: monthly totals in thousands. Jan 49 ? Dec 60":"Sales"})

In [9]:
# Convert the month column to a datetime object
df['Month'] = pd.to_datetime(df['Month'])

# Set the month column as the index
df.set_index('Month', inplace=True)

In [10]:
df

,Sales
Month,
1949-01-01,112
1949-02-01,118
1949-03-01,132
1949-04-01,129
1949-05-01,121
...,...
1960-08-01,606
1960-09-01,508
1960-10-01,461


#checking for stationarity
Stationarity is an important assumption for many time series forecasting models. A stationary time series has constant statistical properties over time, such as mean, variance, and autocovariance. 
If a time series is not stationary, its statistical properties change over time, making it difficult to model and predict future values. By checking for stationarity, you can identify trends, seasonality, and other patterns in the data that can be addressed through transformations, such as differencing or detrending, to make the time series stationary and suitable for forecasting.

In [12]:
#kpss test for stationarity
def check_stationarity(series):
  statistics,p_value,n_lags,critical_values = sm.tsa.stattools.kpss(series)
  print(f'p value:{p_value}')
  print(f'Result: The series is {"not " if p_value < 0.05 else ""} stationary \n')
check_stationarity(df['Sales'])

p value:0.01
Result: The series is not  stationary 



/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/stattools.py:1875: FutureWarning: The behavior of using nlags=None will change in release 0.13.Currently nlags=None is the same as nlags="legacy", and so a sample-size lag length is used. After the next release, the default will change to be the same as nlags="auto" which uses an automatic lag length selection method. To silence this warning, either use "auto" or "legacy"
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/stattools.py:1906: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  warnings.warn(


There are several methods to check for stationarity in a time series:

Plotting Rolling Statistics: Plotting the moving mean and moving standard deviation of the time series can help you identify stationarity. A stationary time series will have a constant mean and constant standard deviation over time.

Dickey-Fuller Test: This is a statistical test that checks for stationarity in a time series. The null hypothesis of the test is that the time series is non-stationary. If the test statistic is less than the critical value, you can reject the null hypothesis and conclude that the time series is stationary.

Augmented Dickey-Fuller Test: This is an extension of the Dickey-Fuller test that allows for the presence of autoregressive terms in the time series.

If the time series is not stationary, you can make it stationary by using various transformations, such as:

Differencing: Subtracting the time series values from their previous values (first differences) or subtracting the first differences from each other (second differences) can make the time series stationary.

Detrending: Removing a linear or non-linear trend from the time series can make it stationary.

Seasonal Adjustment: Removing the seasonal component of the time series can make it stationary.

Transformation: Taking the log, square root, or other transformations of the time series can make it stationary.

In [13]:
import warnings
warnings.filterwarnings("ignore")

The method for forecasting in time series depends on the characteristics of the data, such as stationarity, trend, seasonality, etc. Some popular methods for time series forecasting include:

ARIMA (AutoRegressive Integrated Moving Average)
Exponential Smoothing
Holt-Winters Method
SARIMA (Seasonal ARIMA)
Prophet
LSTM (Long Short-Term Memory)
ARIMAX (AutoRegressive Integrated Moving Average with exogenous variables)

The choice of method will depend on the specific requirements of your problem, and may involve exploring different methods and tuning the parameters to obtain the best results.

In [20]:
#here I have done forecasting with ARMA model
# Import the necessary libraries
from statsmodels.tsa.arima_model import ARMA


# Split the data into training and testing sets
train = df[:'1960-12-01']
test = df['1961-01-01':]

# Fit the ARMA model
model = ARMA(train, order=(1,1))
model_fit = model.fit()

# Predict the next quarter of 1961
predictions = model_fit.predict(start='1961-01-01', end='1961-03-01')

In [21]:
predictions

1961-01-01    453.899744
1961-02-01    443.089597
1961-03-01    432.957038
Freq: MS, dtype: float64

In [22]:
# Obtain the AIC and BIC
print('AIC: ', model_fit.aic)
print('BIC: ', model_fit.bic)

AIC:  1409.7482700864462
BIC:  1421.6275232847502
